In [1]:
from openai import OpenAI

In [2]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [3]:
chat_response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about Samuel L. Jackson."},
    ]
)

In [4]:
print("Chat response:", chat_response)
print("Generated response:", chat_response.choices[0].message.content)

Chat response: ChatCompletion(id='chatcmpl-8bd37a8503de41419b58626a796a0cfe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Samuel L. Jackson! One of the most iconic and beloved actors of our time. Here are some fascinating facts about this talented individual:\n\n**Early Life**\n\nSamuel Leroy Jackson was born on December 21, 1948, in Washington, D.C. He grew up in Chattanooga, Tennessee, and was raised by his mother, Elizabeth, a factory worker, and his maternal grandparents. Jackson attended Morehouse College in Atlanta, Georgia, where he became involved in the Black Power movement and was a member of the Omega Psi Phi fraternity.\n\n**Career**\n\nJackson began his acting career in the early 1970s, appearing in various stage productions and small film roles. His breakthrough came in 1991 with Spike Lee\'s film "Jungle Fever," which earned him a special jury prize for best supporting actor at the Cannes Film Festival.\n\nSome of 

In [5]:
DATASET_PATH = "/net/projects/veitch/prompt_distributions/data/rewrites/helpsteer_complexity_dataset_20240921_213510.json"

In [6]:
import json

with open(DATASET_PATH, 'r') as f:
    dataset = json.load(f)

In [7]:
dataset['0']

{'w_original': False,
 'completions': {'original': "To build an assistive device to help an elderly person with basic needs, one must consider three crucial things: safety, compatibility, and ease of use. Safety is paramount, as the device must not cause harm to the user. Compatibility with the user's environment and other devices is also essential. Finally, the device must be simple enough for the elderly person to operate.",
  'rewrite': "When designing an assistive apparatus intended to aid an elderly individual in fulfilling fundamental requirements, three pivotal considerations must be meticulously evaluated: safety, compatibility, and ease of use. Safety takes precedence as the foremost concern, ensuring that the device does not inadvertently pose any risk or harm to the user. Equally important is the device's compatibility, encompassing seamless integration within the user's existing environment and interoperability with other technological devices they may utilize. Lastly, the 

In [ ]:
prompt = "{text} Please rate the above text as either {w_1} or {w_0}. Don't add any additional information: \n A: {w_1}\n B: {w_0} \nAnswer: "

w_1 = "complex"
w_0 = "not complex"

n_examples = 10
correct = 0
failure_to_answer = 0
for value in list(dataset.values())[:n_examples]:
    correct_answer = "A" if not value['w_original'] else "B"
    chat_response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-70B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt.format(text=value['completions']['rewrite'], w_1=w_1, w_0=w_0)},
        ],
        max_tokens=1,
    )
    generated_text = chat_response.choices[0].message.content
    model_answer = generated_text[-1]
    print(generated_text)
    if model_answer == correct_answer:
        correct += 1
    if model_answer not in {"A", "B"}:
        failure_to_answer += 1
print("Accuracy: ", correct/(n_examples - failure_to_answer))


A
Accuracy:  0.01
